In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 300)
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# import seaborn as sns
from catboost import CatBoostRegressor, Pool
import optuna

In [2]:
# pickle (train df after feature engineering) is located in my private dataset
path = "../input/fe-train-test-data/"
train = pd.read_pickle(path + 'train.pkl')
test = pd.read_pickle(path + 'test.pkl')

In [3]:
train.head(10)

,stock_id,time_id,target,row_id,wap1_sum,wap1_std,wap2_sum,wap2_std,wap3_sum,wap3_std,wap4_sum,wap4_std,log_return1_realized_volatility,log_return2_realized_volatility,log_return3_realized_volatility,log_return4_realized_volatility,wap_balance_sum,wap_balance_amax,price_spread_sum,price_spread_amax,price_spread2_sum,price_spread2_amax,bid_spread_sum,bid_spread_amax,ask_spread_sum,ask_spread_amax,total_volume_sum,total_volume_amax,volume_imbalance_sum,volume_imbalance_amax,bid_ask_spread_sum,bid_ask_spread_amax,log_return1_realized_volatility_500,log_return2_realized_volatility_500,log_return3_realized_volatility_500,log_return4_realized_volatility_500,log_return1_realized_volatility_400,log_return2_realized_volatility_400,log_return3_realized_volatility_400,log_return4_realized_volatility_400,log_return1_realized_volatility_300,log_return2_realized_volatility_300,log_return3_realized_volatility_300,log_return4_realized_volatility_300,log_return1_realized_volatility_200,log_return2_realized_volatility_200,log_return3_realized_volatility_200,log_return4_realized_volatility_200,log_return1_realized_volatility_100,log_return2_realized_volatility_100,log_return3_realized_volatility_100,log_return4_realized_volatility_100,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_size_amax,trade_size_amin,trade_order_count_sum,trade_order_count_amax,trade_amount_sum,trade_amount_amax,trade_amount_amin,trade_tendency,trade_f_max,trade_f_min,trade_df_max,trade_df_min,trade_abs_diff,trade_energy,trade_iqr_p,trade_abs_diff_v,trade_energy_v,trade_iqr_p_v,trade_log_return_realized_volatility_500,trade_seconds_in_bucket_count_unique_500,trade_size_sum_500,trade_order_count_sum_500,trade_log_return_realized_volatility_400,trade_seconds_in_bucket_count_unique_400,trade_size_sum_400,trade_order_count_sum_400,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_sum_300,trade_log_return_realized_volatility_200,trade_seconds_in_bucket_count_unique_200,trade_size_sum_200,trade_order_count_sum_200,trade_log_return_realized_volatility_100,trade_seconds_in_bucket_count_unique_100,trade_size_sum_100,trade_order_count_sum_100,log_return1_realized_volatility_mean_stock,log_return1_realized_volatility_std_stock,log_return1_realized_volatility_max_stock,log_return1_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return1_realized_volatility_400_mean_stock,log_return1_realized_volatility_400_std_stock,log_return1_realized_volatility_400_max_stock,log_return1_realized_volatility_400_min_stock,log_return2_realized_volatility_400_mean_stock,log_return2_realized_volatility_400_std_stock,log_return2_realized_volatility_400_max_stock,log_return2_realized_volatility_400_min_stock,log_return1_realized_volatility_300_mean_stock,log_return1_realized_volatility_300_std_stock,log_return1_realized_volatility_300_max_stock,log_return1_realized_volatility_300_min_stock,log_return2_realized_volatility_300_mean_stock,log_return2_realized_volatility_300_std_stock,log_return2_realized_volatility_300_max_stock,log_return2_realized_volatility_300_min_stock,log_return1_realized_volatility_200_mean_stock,log_return1_realized_volatility_200_std_stock,log_return1_realized_volatility_200_max_stock,log_return1_realized_volatility_200_min_stock,log_return2_realized_volatility_200_mean_stock,log_return2_realized_volatility_200_std_stock,log_return2_realized_volatility_200_max_stock,log_return2_realized_volatility_200_min_stock,trade_log_return_realized_volatility_mean_stock,trade_log_return_realized_volatility_std_stock,trade_log_return_realized_volatility_max_stock,trade_log_return_realized_volatility_min_stock,trade_log_return_realized_volatility_400_mean_stock,trade_log_return_realized_volatility_400_std_stock,trade_log_return_re

In [4]:
train.shape

(428932, 248)

In [5]:
## N_TRIALS = 100

# TIME => studyする時間を秒で指定 (この時間内で確認できたbest paramをreturn)
# TIME = 1800 => 1800 seconds (60 * 30)
# kaggle_max => 60 * 60 * 9.0 => 時間
# TIME = 60 * 60 * 8.80
TIME = 60 * 60 * 8.80
N_SPLITS = 5
RANDOM_STATE = 99
kfold = KFold(N_SPLITS, random_state=RANDOM_STATE, shuffle=True)

FIXED_PARAMS = {
                "metric" : "None",
                'verbosity': -1,
                'n_jobs': -1,
                'seed': RANDOM_STATE}

def rmspe(y_true, y_pred):
        return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [6]:
X = train
y = test

In [7]:
def objective(trial):
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.33, random_state=99, shuffle=True)
    train_pool = Pool(train_x, train_y)
    test_pool = Pool(test_x, test_y)

    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }
    rmspe_list = []

    # Learning
    model = CatBoostRegressor(eval_metric="RMSE", random_seed=99, **params)
    model.fit(train_pool)
    # Predict
    preds = model.predict(test_pool)
    pred_labels = np.rint(preds)
    # Evaluation
    score = rmspe(test_y, preds)
    rmspe_list.append(score)
    return np.mean(rmspe_list)

In [8]:
# def objective(trial):
    
#     params = {
#         'n_estimators' : trial.suggest_int('iterations', 500, 2000),
#         'iterations' : trial.suggest_int('iterations', 50, 300),                         
#         'depth' : trial.suggest_int('depth', 4, 10),                                       
#         'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
#         'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
#         'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'od_wait' :trial.suggest_int('od_wait', 10, 50)
#     }
    
#     params.update(FIXED_PARAMS)
    
#     # トレーニングデータとテストデータを分割
#     train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
#     train_pool = Pool(train_x, train_y, cat_features=categorical_features_indices)
#     test_pool = Pool(test_x, test_y, cat_features=categorical_features_indices)


#     pruning_callback = optuna.integration.CatBoostPruningCallback(
#         trial, "RMSPE", valid_name='valid_1')
#     rmspe_list = []
    
# #     for kfold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
# #         X_train = X.iloc[train_idx]
# #         y_train = y.iloc[train_idx]
# #         X_val = X.iloc[val_idx]
# #         y_val = y.iloc[val_idx]

# #         d_train = lgb.Dataset(X_train, label=y_train)
# #         d_valid = lgb.Dataset(X_val, label=y_val)

# #         model = lgb.train(params,
# #                       train_set=d_train,
# #                       valid_sets=[d_train, d_valid],
# #                       verbose_eval=0,
# #                       early_stopping_rounds=100,
# #                       callbacks=[pruning_callback],
# #                       feval = feval_rmspe)

# #         preds = model.predict(X_val)
# #         score = rmspe(y_val, preds)
        
# #         rmspe_list.append(score)
    
# #     return np.mean(rmspe_list)

In [9]:
X_display = train.drop(['row_id', 'time_id', 'target'], axis = 1)
X = X_display
y = train['target']

In [10]:
study = optuna.create_study(
    direction='minimize', 
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=25))
study.optimize(objective, timeout=TIME)
    
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-09-27 03:23:51,536] A new study created in memory with name: no-name-5e0e006f-bd40-4535-9e01-d0f801ffdda2


0:	learn: 0.0028739	total: 765ms	remaining: 45.9s
1:	learn: 0.0028234	total: 1.24s	remaining: 36.7s
2:	learn: 0.0027711	total: 1.71s	remaining: 33s
3:	learn: 0.0027224	total: 2.1s	remaining: 29.9s
4:	learn: 0.0026714	total: 2.52s	remaining: 28.2s
5:	learn: 0.0026221	total: 2.93s	remaining: 26.9s
6:	learn: 0.0025757	total: 3.31s	remaining: 25.6s
7:	learn: 0.0025333	total: 3.68s	remaining: 24.4s
8:	learn: 0.0024901	total: 4.06s	remaining: 23.5s
9:	learn: 0.0024475	total: 4.45s	remaining: 22.7s
10:	learn: 0.0024068	total: 4.84s	remaining: 22s
11:	learn: 0.0023662	total: 5.18s	remaining: 21.2s
12:	learn: 0.0023282	total: 5.58s	remaining: 20.6s
13:	learn: 0.0022905	total: 5.97s	remaining: 20s
14:	learn: 0.0022532	total: 6.32s	remaining: 19.4s
15:	learn: 0.0022176	total: 6.69s	remaining: 18.8s
16:	learn: 0.0021840	total: 7.05s	remaining: 18.3s
17:	learn: 0.0021506	total: 7.43s	remaining: 17.7s
18:	learn: 0.0021164	total: 7.77s	remaining: 17.2s
19:	learn: 0.0020863	total: 8.13s	remaining: 16.

[I 2021-09-27 03:24:19,305] Trial 0 finished with value: 0.4349058746551738 and parameters: {'iterations': 61, 'depth': 9, 'learning_rate': 0.028158889097460404, 'random_strength': 41, 'bagging_temperature': 4.225414536115269, 'od_type': 'Iter', 'od_wait': 41}. Best is trial 0 with value: 0.4349058746551738.


60:	learn: 0.0014099	total: 23.5s	remaining: 0us
0:	learn: 0.0028857	total: 445ms	remaining: 1m 22s
1:	learn: 0.0028451	total: 828ms	remaining: 1m 16s
2:	learn: 0.0028027	total: 1.23s	remaining: 1m 15s
3:	learn: 0.0027630	total: 1.63s	remaining: 1m 14s
4:	learn: 0.0027213	total: 2.01s	remaining: 1m 13s
5:	learn: 0.0026807	total: 2.4s	remaining: 1m 12s
6:	learn: 0.0026422	total: 2.78s	remaining: 1m 11s
7:	learn: 0.0026068	total: 3.15s	remaining: 1m 10s
8:	learn: 0.0025706	total: 3.52s	remaining: 1m 9s
9:	learn: 0.0025389	total: 3.92s	remaining: 1m 9s
10:	learn: 0.0025042	total: 4.32s	remaining: 1m 9s
11:	learn: 0.0024694	total: 4.66s	remaining: 1m 7s
12:	learn: 0.0024366	total: 5.1s	remaining: 1m 8s
13:	learn: 0.0024071	total: 5.61s	remaining: 1m 9s
14:	learn: 0.0023742	total: 6.07s	remaining: 1m 9s
15:	learn: 0.0023427	total: 6.5s	remaining: 1m 9s
16:	learn: 0.0023127	total: 6.86s	remaining: 1m 8s
17:	learn: 0.0022831	total: 7.28s	remaining: 1m 8s
18:	learn: 0.0022524	total: 7.65s	rema

[I 2021-09-27 03:25:35,302] Trial 1 finished with value: 0.29324765985095036 and parameters: {'iterations': 187, 'depth': 9, 'learning_rate': 0.022484839769706687, 'random_strength': 43, 'bagging_temperature': 23.65211284469275, 'od_type': 'IncToDec', 'od_wait': 33}. Best is trial 1 with value: 0.29324765985095036.


...

0:	learn: 0.0023626	total: 1.2s	remaining: 5m 51s
1:	learn: 0.0019679	total: 2.24s	remaining: 5m 26s
2:	learn: 0.0017034	total: 3.15s	remaining: 5m 5s
3:	learn: 0.0015241	total: 4.09s	remaining: 4m 56s
4:	learn: 0.0014029	total: 5.04s	remaining: 4m 51s
5:	learn: 0.0013215	total: 6.02s	remaining: 4m 49s
6:	learn: 0.0012675	total: 6.95s	remaining: 4m 45s
7:	learn: 0.0012303	total: 7.87s	remaining: 4m 41s
8:	learn: 0.0011988	total: 8.81s	remaining: 4m 38s
9:	learn: 0.0011813	total: 9.75s	remaining: 4m 36s
10:	learn: 0.0011617	total: 10.7s	remaining: 4m 34s
11:	learn: 0.0011483	total: 11.6s	remaining: 4m 32s
12:	learn: 0.0011384	total: 12.6s	remaining: 4m 31s
13:	learn: 0.0011307	total: 13.5s	remaining: 4m 30s
14:	learn: 0.0011231	total: 14.5s	remaining: 4m 29s
15:	learn: 0.0011189	total: 15.4s	remaining: 4m 27s
16:	learn: 0.0011077	total: 16.4s	remaining: 4m 27s
17:	learn: 0.0011010	total: 17.3s	remaining: 4m 25s
18:	learn: 0.0010937	total: 18.3s	remaining: 4m 24s
19:	learn: 0.0010900	tot

[I 2021-09-27 12:11:05,253] Trial 152 finished with value: 0.22610239557188752 and parameters: {'iterations': 294, 'depth': 10, 'learning_rate': 0.2628985011774806, 'random_strength': 3, 'bagging_temperature': 44.97111826194773, 'od_type': 'IncToDec', 'od_wait': 14}. Best is trial 145 with value: 0.22282784714706652.


0:	learn: 0.0023833	total: 929ms	remaining: 4m 33s
1:	learn: 0.0019917	total: 1.86s	remaining: 4m 33s
2:	learn: 0.0017315	total: 2.79s	remaining: 4m 31s
3:	learn: 0.0015513	total: 3.73s	remaining: 4m 31s
4:	learn: 0.0014278	total: 4.67s	remaining: 4m 31s
5:	learn: 0.0013478	total: 5.71s	remaining: 4m 35s
6:	learn: 0.0012936	total: 6.69s	remaining: 4m 35s
7:	learn: 0.0012514	total: 7.91s	remaining: 4m 43s
8:	learn: 0.0012190	total: 8.85s	remaining: 4m 41s
9:	learn: 0.0012018	total: 9.95s	remaining: 4m 43s
10:	learn: 0.0011801	total: 11.2s	remaining: 4m 48s
11:	learn: 0.0011679	total: 12.4s	remaining: 4m 52s
12:	learn: 0.0011511	total: 13.6s	remaining: 4m 55s
13:	learn: 0.0011407	total: 14.8s	remaining: 4m 57s
14:	learn: 0.0011317	total: 16.1s	remaining: 5m
15:	learn: 0.0011239	total: 17.4s	remaining: 5m 2s
16:	learn: 0.0011176	total: 18.6s	remaining: 5m 4s
17:	learn: 0.0011138	total: 19.8s	remaining: 5m 4s
18:	learn: 0.0011043	total: 20.9s	remaining: 5m 3s
19:	learn: 0.0011001	total: 22

[I 2021-09-27 12:16:04,235] Trial 153 finished with value: 0.22674843279826426 and parameters: {'iterations': 295, 'depth': 10, 'learning_rate': 0.25424958014540217, 'random_strength': 4, 'bagging_temperature': 48.64044640286811, 'od_type': 'IncToDec', 'od_wait': 15}. Best is trial 145 with value: 0.22282784714706652.


Number of finished trials: 154
Best trial: {'iterations': 283, 'depth': 10, 'learning_rate': 0.29749360260606733, 'random_strength': 0, 'bagging_temperature': 50.343747762899255, 'od_type': 'IncToDec', 'od_wait': 18}
